In [24]:
from PIL import Image
import cv2
import os 
import json
import random 

In [17]:
def resize_images_in_folder(folder_path, target_size):

    for filename in os.listdir(folder_path):
        # Construct the full path
        file_path = os.path.join(folder_path, filename)

        if filename.endswith((".jpg")):
            try:
                with Image.open(file_path) as img:
                    # Resize the image
                    img_resized = img.resize(target_size)

                    # Override the resized img
                    img_resized.save(file_path)

            except Exception as e:
                print("ERRROR")

In [18]:
path_to_folder = "../university_annotations/train/train_images"
target_size = (320, 240)
resize_images_in_folder(path_to_folder, target_size)
print("Your task has been completed MASTER")

Your task has been completed MASTER


In [19]:
# Selecting a random image to see if has the shape required
random_file = random.choice(os.listdir(path_to_folder))
random_file_path = os.path.join(path_to_folder, random_file)
image = cv2.imread(random_file_path)
image.shape

(240, 320, 3)

In [28]:
# Changing the shape inside the coco dataset 
coco_json_path = "../university_annotations/train/_annotations.coco.json"
new_coco_json_path = "../university_annotations/train/new_annotations.coco.json"

# Target size (320, 240)
target_width = 320
target_height = 240

# Load the original COCO dataset
with open(coco_json_path, 'r') as f:
    coco_data = json.load(f)

# Function to adjust annotations for the new image size
def adjust_annotations(coco_data, target_width, target_height):
    for image_info in coco_data['images']:
        original_width, original_height = image_info['width'], image_info['height']
        
        # Update the image size in the COCO data
        image_info['width'], image_info['height'] = target_width, target_height

        # Calculate scaling factors
        x_scale = target_width / original_width
        y_scale = target_height / original_height

        # Adjust the annotations for the resized image
        for annotation in coco_data['annotations']:
            if annotation['image_id'] == image_info['id']:
                bbox = annotation['bbox']
                
                # Adjust the bounding box
                bbox[0] *= x_scale  # x coordinate
                bbox[1] *= y_scale  # y coordinate
                bbox[2] *= x_scale  # width
                bbox[3] *= y_scale  # height
                
    return coco_data

# Adjust the annotations for the new image size
new_coco_data = adjust_annotations(coco_data, target_width, target_height)

# Save the new COCO dataset
with open(new_coco_json_path, 'w') as f:
    json.dump(new_coco_data, f)

print(f"Annotations have been refactored and saved to {new_coco_json_path}")


Annotations have been refactored and saved to ../university_annotations/train/new_annotations.coco.json


In [32]:
# Paths
images_folder = '../university_annotations/train/train_images'
coco_json_path = '../university_annotations/train/new_annotations.coco.json'
output_image_path = '../university_annotations/train/labeled_image.jpg'

# Load the COCO dataset
with open(coco_json_path, 'r') as f:
    coco_data = json.load(f)

# Select a random image
random_image_info = random.choice(coco_data['images'])
random_image_filename = random_image_info['file_name']
random_image_path = os.path.join(images_folder, random_image_filename)

# Load the image
image = cv2.imread(random_image_path)

# Get the annotations for the selected image
image_id = random_image_info['id']
annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]

# Load category names
categories = {cat['id']: cat['name'] for cat in coco_data['categories']}

# Draw bounding boxes and labels
for annotation in annotations:
    bbox = annotation['bbox']
    category_id = annotation['category_id']
    label = categories[category_id]
    
    # Convert bbox to integers
    x, y, w, h = map(int, bbox)
    
    # Draw the bounding box
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Put the label above the bounding box
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

# Save the result to a file
cv2.imwrite(output_image_path, image)

print(f"Image with bounding boxes saved to {output_image_path}")


Image with bounding boxes saved to ../university_annotations/train/labeled_image.jpg
